In [13]:
import random

In [14]:
import os, glob, shutil

In [16]:
from tqdm import tqdm

In [1]:
import torch

In [14]:
a = torch.arange(27).reshape(3, 3, 3)
a

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])

In [29]:
d = {x: x * 10 for x in range(10)}

In [31]:
for key, value in d.items():
    if value > 50:
        d[key] = value + 1

In [34]:
import numpy as np

a = np.arange(10)
b = np.arange(10) * 100

In [64]:
import cc3d

In [65]:
a = np.arange(27).reshape(3, 3, 3)
a[a < 12] = 100
a

array([[[100, 100, 100],
        [100, 100, 100],
        [100, 100, 100]],

       [[100, 100, 100],
        [ 12,  13,  14],
        [ 15,  16,  17]],

       [[ 18,  19,  20],
        [ 21,  22,  23],
        [ 24,  25,  26]]])

In [66]:
macro_components, num = cc3d.connected_components(
    a,
    connectivity=26,
    return_N=True,
    out_dtype=np.uint16,
)

In [68]:
len(components_volumes)

17

In [79]:
bin(1 << 3).count("1")

1

In [70]:
import numpy as np
import scipy.ndimage as ndimage


def compute_composite_mask(tensor):
    """
    Convert a 4D tensor (channels, n, n, n) into a 3D composite volume.
    Each voxel value is computed as the sum of (1 << channel) for each channel
    where the voxel has a 1.
    """
    # Get spatial dimensions from tensor (assumes tensor.shape = (channels, n, n, n))
    composite = np.zeros(tensor.shape[1:], dtype=np.int32)
    num_channels = tensor.shape[0]
    for channel in range(num_channels):
        # For each channel, add the bit (1 << channel) to the composite mask if voxel is 1.
        composite += tensor[channel] * (1 << channel)
    return composite


def get_intersection_labels(composite):
    """
    Return composite labels corresponding to intersections (i.e. where at least 2 classes overlap).
    This is determined by counting the number of 1 bits in the composite label.
    """
    unique_labels = np.unique(composite)
    # We only want labels that represent 2 or more classes (i.e. popcount >= 2)
    intersection_labels = [
        label for label in unique_labels if bin(label).count("1") >= 2
    ]
    return intersection_labels


def connected_components_for_label(composite, label, connectivity=26):
    """
    For a given composite label, run a 3D connected components analysis.
    Uses a 3x3x3 connectivity (26-connected) if requested.
    """
    # Create a boolean mask of the voxels that have exactly this composite label.
    mask = composite == label
    # Define the connectivity structure.
    if connectivity == 26:
        structure = np.ones((3, 3, 3), dtype=np.int32)
    else:
        # For 6-connected, use:
        structure = ndimage.generate_binary_structure(3, 1)
    labeled_array, num_features = ndimage.label(mask, structure=structure)
    return labeled_array, num_features


def decode_label(label, mapping):
    """
    Given an integer composite label and the mapping dictionary (class name -> channel index),
    return a list of class names that are present in the label.
    """
    present_classes = []
    for class_name, idx in mapping.items():
        if label & (1 << idx):
            present_classes.append(class_name)
    return present_classes


# ----- Example and Test -----
if __name__ == "__main__":
    # mapping: keys are class names, values are the channel indices in the 4D tensor.
    mapping = {"class1": 0, "class2": 1, "class3": 2}

    # Create a 4D tensor of shape (channels, n, n, n) with binary masks.
    # Here we use a 10x10x10 volume and 3 classes.
    tensor = np.zeros((3, 10, 10, 10), dtype=np.uint8)

    # For class1, draw a cube from indices [2:7, 2:7, 2:7]
    tensor[mapping["class1"], 2:7, 2:7, 2:7] = 1

    # For class2, draw a cube from indices [5:9, 5:9, 5:9] so it overlaps with class1 in [5:7, 5:7, 5:7]
    tensor[mapping["class2"], 5:9, 5:9, 5:9] = 1

    # For class3, first draw a cube from indices [0:4, 0:4, 0:4] (non-overlapping with class1/class2)
    tensor[mapping["class3"], 0:4, 0:4, 0:4] = 1

    # Then, create an additional region for class3 that overlaps with class2
    tensor[mapping["class3"], 4:8, 4:8, 4:8] = 1

    # Compute the composite mask where each voxel encodes which classes are present.
    composite = compute_composite_mask(tensor)

    # Identify the composite labels that represent intersections (2 or more classes)
    intersection_labels = get_intersection_labels(composite)
    print("Intersection composite labels found:", intersection_labels)

    # For each intersection type, run connected components analysis.
    for label in intersection_labels:
        labeled_array, num_components = connected_components_for_label(
            composite, label, connectivity=26
        )
        classes = decode_label(label, mapping)
        print(
            f"Composite label {label} (classes: {classes}) has {num_components} connected component(s)"
        )

Intersection composite labels found: [5, 6, 7]
Composite label 5 (classes: ['class1', 'class3']) has 1 connected component(s)
Composite label 6 (classes: ['class2', 'class3']) has 1 connected component(s)
Composite label 7 (classes: ['class1', 'class2', 'class3']) has 1 connected component(s)


In [80]:
a = np.arange(27).reshape(3, 3, 3)
a[a > 5].shape

(21,)

In [83]:
lst = [[1, 2], [3, 4]]

In [88]:
lst = [x for x in range(10)]
while len(lst) > 5:
    print("ahh")
    print(lst.pop())

ahh
9
ahh
8
ahh
7
ahh
6
ahh
5


In [91]:
def _find_connected_components(arr):
    return cc3d.connected_components(
        arr,
        connectivity=26,
        return_N=True,
        out_dtype=np.uint16,
    )


def _get_components_and_ids(macro_components):
    for component_id, component_mask in cc3d.each(
        macro_components, binary=True, in_place=True
    ):
        yield component_id, component_mask

In [1]:
import cc3d
import numpy as np

In [50]:
a = np.arange(27).reshape(3, 3, 3)
a

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [51]:
a = np.arange(27).reshape(3, 3, 3)
a[a != 7] = 0

In [57]:
b = np.arange(27).reshape(3, 3, 3)
b[b != 26] = 0

In [60]:
def _find_connected_components(arr):
    return cc3d.connected_components(
        arr,
        connectivity=26,
        return_N=True,
        out_dtype=np.uint16,
    )


def _get_surface_contact(mask, mask2=None):
    if mask2 is not None:
        two_component_mask = mask + mask2
    else:
        two_component_mask = (mask > 0).astype(int)
        two_component_mask[two_component_mask == 0] = -1
    components, _ = _find_connected_components(two_component_mask)
    surface_per_contact = cc3d.contacts(
        components,
        connectivity=26,
    )

    return surface_per_contact[(1, 2)] if len(surface_per_contact) > 0 else 0

In [68]:
def _get_centroid(mask):
    coords = np.argwhere(mask == 1)
    # Compute the centroid
    return coords.mean(axis=0)

In [9]:
!pip install scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 11.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [36]:
import numpy as np
from scipy.spatial import ConvexHull
from skimage.morphology import convex_hull_image
from skimage.measure import regionprops, label


def compute_convex_hull_filled(mask):
    dims = len(mask.shape)
    # Get coordinates of the object (where mask == 1)
    points = np.column_stack(np.where(mask > 0))

    # Compute convex hull
    hull = ConvexHull(points)
    print(hull)

    # Create a new mask with the convex hull points
    hull_mask = np.zeros_like(mask, dtype=bool)

    # Fill the convex hull using the bounding box
    min_coords = points.min(axis=0)
    max_coords = points.max(axis=0)

    # Create an empty mask in the bounding box region
    bbox = tuple(slice(min_coords[d], max_coords[d] + 1) for d in range(dims))

    # Convert hull vertices back to an image mask
    hull_points = points[hull.vertices]
    hull_mask[tuple(hull_points.T)] = True

    # Fill the convex hull in the bounding box region
    filled_hull = convex_hull_image(hull_mask[bbox])

    # Put back into the original image
    hull_mask[bbox] = filled_hull

    return hull_mask.astype(np.uint8)


# Example: Create a 2D non-convex mask (5x5 "L" shape)
mask = np.zeros((5, 5), dtype=np.uint8)
mask[1:4, 1] = 1  # Vertical part of L
mask[3, 2:4] = 1  # Horizontal part of L

convex_hull_mask = compute_convex_hull_filled(mask)

In [37]:
mask = np.repeat(mask[None, :, :], 5, axis=0)

In [38]:
mask.shape

(5, 5, 5)

In [42]:
a = np.array([0, 1])
b = np.array([10, 10])
np.sum([a, b])

21

In [39]:
def _mask2bbox(mask):
    dims = len(mask.shape)
    points = np.column_stack(np.where(mask > 0))
    min_coords = points.min(axis=0)
    max_coords = points.max(axis=0)

    res = np.zeros_like(mask)
    bbox = tuple(slice(min_coords[d], max_coords[d] + 1) for d in range(dims))

    res[bbox] = 1
    return res

In [40]:
_mask2bbox(mask)

array([[[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]]], dtype=uint8)

In [27]:
points = np.column_stack(np.where(mask > 0))
points

array([[1, 1],
       [2, 1],
       [3, 1],
       [3, 2],
       [3, 3]])

In [25]:
convex_hull_image(np.arange(27).reshape(3, 3, 3))

array([[[False,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]])

In [24]:
convex_hull_image(mask).astype(int)

array([[0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]])

In [20]:
mask

array([[0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

In [19]:
convex_hull_mask

array([[0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

In [7]:
a = {1: 2, 3: 4}
sum(a.values())

6

In [6]:
lst = []
lst.append((1, 2))

In [2]:
import numpy as np

In [3]:
a = np.arange(27).reshape(3, 3, 3)

In [4]:
a[2]

array([[18, 19, 20],
       [21, 22, 23],
       [24, 25, 26]])

In [61]:
_get_surface_contact(a, b)

0

In [20]:
components, num = cc3d.connected_components(
    a,
    connectivity=26,
    return_N=True,
    # out_dtype=np.uint16,
)

In [21]:
components

array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 2, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]], dtype=uint16)

In [28]:
surface_per_contact = cc3d.contacts(
    components,
    connectivity=26,
)

In [29]:
surface_per_contact

{(1, 2): 40.0}

In [8]:
num

1

In [89]:
lst

[0, 1, 2, 3, 4]

In [86]:
[1, 2] + [3, 4]

[1, 2, 3, 4]

In [87]:
sum(lst)

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [69]:
import numpy as np
import cc3d


def slice_largest_component_in_half(component, axis=0, side="left", connectivity=6):
    """
    Extracts the largest connected component from a binary array and slices it along a specified
    spatial axis, keeping only one half of the component.

    This function supports both 3D arrays (shape: n x n x n) and 4D arrays with a single channel
    (shape: 1 x n x n x n). For a 4D array, the channel dimension is ignored during processing and
    then reinserted before returning the result.

    Parameters:
        component : np.ndarray
            A binary numpy array. Expected shape is (n, n, n) or (1, n, n, n).
        axis : int, optional
            The spatial axis along which to slice (0, 1, or 2 for 3D data). Default is 0.
        side : str, optional
            Which side to retain; 'left' keeps voxels with coordinate <= the slice,
            while 'right' keeps those > the slice. Default is 'left'.
        connectivity : int, optional
            The connectivity for cc3d (e.g. 6, 18, or 26). Default is 6.

    Returns:
        np.ndarray: A binary numpy array (with the same shape as the input) containing the sliced
                    largest connected component.
    """
    # If the array has a channel dimension (shape: 1, n, n, n), extract the spatial data.
    has_channel = component.ndim == 4 and component.shape[0] == 1
    if has_channel:
        data = component[0]
    else:
        data = component

    # Extract connected components using cc3d.
    labeled, num_labels = cc3d.connected_components(
        data, connectivity=connectivity, return_N=True
    )
    if num_labels == 0:
        raise ValueError("No connected components found in the input array.")

    # Determine the largest component (ignore the background label 0).
    counts = np.bincount(labeled.flat)
    counts[0] = 0  # background should not be considered
    largest_label = counts.argmax()
    largest_component = (labeled == largest_label).astype(data.dtype)

    # Find the bounding box of the largest component.
    coords = np.argwhere(largest_component)
    if coords.size == 0:
        raise ValueError("The largest component is empty.")
    min_coords = coords.min(axis=0)
    max_coords = coords.max(axis=0)

    # Choose a slicing coordinate roughly in the center along the specified axis.
    slice_coord = (min_coords[axis] + max_coords[axis]) // 2

    # Build a mask that retains only one half of the component.
    grid = np.indices(largest_component.shape)[axis]
    if side == "left":
        mask = grid <= slice_coord
    elif side == "right":
        mask = grid > slice_coord
    else:
        raise ValueError("Parameter 'side' must be either 'left' or 'right'.")

    # Apply the mask to the largest component.
    sliced_component = largest_component * mask

    # Reinsert the channel dimension if the original array had one.
    if has_channel:
        sliced_component = sliced_component[np.newaxis, ...]

    return sliced_component


# Example usage:
if __name__ == "__main__":
    # Create a simple 4D binary array with one channel.
    arr = np.zeros((1, 20, 20, 20), dtype=int)
    # Create two connected components: a large one and a small one.
    arr[0, 4:16, 4:16, 4:16] = 1  # large component
    arr[0, 0:3, 0:3, 0:3] = 1  # small component

    # Extract the largest component and slice it along spatial axis 0 (keeping the "left" side).
    sliced = slice_largest_component_in_half(arr, axis=0, side="left")

    print("Original total voxels:", arr.sum())
    print("Voxels in sliced largest component:", sliced.sum())

Original total voxels: 1755
Voxels in sliced largest component: 864


In [56]:
max_index

0

In [54]:
for component_id, component_mask in cc3d.each(
    macro_components, binary=True, in_place=True
):
    print(component_id)
    print(component_mask)

1
[[[ True  True  True]
  [ True  True  True]
  [ True  True  True]]

 [[ True  True  True]
  [False False False]
  [False False False]]

 [[False False False]
  [False False False]
  [False False False]]]
2
[[[False False False]
  [False False False]
  [False False False]]

 [[False False False]
  [ True False False]
  [False False False]]

 [[False False False]
  [False False False]
  [False False False]]]
3
[[[False False False]
  [False False False]
  [False False False]]

 [[False False False]
  [False  True False]
  [False False False]]

 [[False False False]
  [False False False]
  [False False False]]]
4
[[[False False False]
  [False False False]
  [False False False]]

 [[False False False]
  [False False  True]
  [False False False]]

 [[False False False]
  [False False False]
  [False False False]]]
5
[[[False False False]
  [False False False]
  [False False False]]

 [[False False False]
  [False False False]
  [ True False False]]

 [[False False False]
  [False False F

In [46]:
np.argmax([3, 2, 1])

0

In [40]:
if None:
    print("a")

In [ ]:
d = {'1': 2}
for key, value in d.values()

In [37]:
new = np.array([a, b])

In [18]:
def foo(ab):
    print("ahh")
    print(ab)
    return True

In [20]:
b = foo(aa := sum(range(5)))

ahh
10


In [21]:
aa

10

In [19]:
if  > 0:
    print(s)

ahh
10
10


In [15]:
list(a)

[tensor([[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]),
 tensor([[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]]),
 tensor([[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]])]

In [11]:
b = torch.eye(3, 3)
b

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [12]:
a[:, b.to(bool)] = 0
a

tensor([[[0., 1., 1.],
         [1., 0., 1.],
         [1., 1., 0.]],

        [[0., 1., 1.],
         [1., 0., 1.],
         [1., 1., 0.]],

        [[0., 1., 1.],
         [1., 0., 1.],
         [1., 1., 0.]]])

In [19]:
for file in tqdm(
    glob.glob("/home/setupishe/datasets/COCO/images/train/*txt", recursive=True)
):
    os.remove(file)

100%|██████████| 118287/118287 [00:00<00:00, 119125.56it/s]


In [1]:
time_05 = 10

In [2]:
speed = time_05 / 0.5

In [6]:
speed

20.0

In [6]:
total_amount = 0.2 + 0.3 + 0.4 + 0.5 + 0.6 + 0.7

In [9]:
total_amount * 200 / 60

9.0

In [11]:
total_amount * speed * 65 / 60 / 24

2.256944444444444

In [7]:
import os, glob, shutil

In [8]:
train_list = []

In [9]:
count = 0
for file in glob.glob("/home/setupishe/datasets/VOC/labels/**/*txt", recursive=True):
    with open(file, "r") as f:
        count += len(f.readlines())

In [10]:
count

52090

In [ ]:
lst = [
    "train2007",
    "train2012",
    "val2007",
    "val2012",
]

train_list = [
    f"./images/{folder}/{x}"
    for folder in lst
    for x in os.listdir(os.path.join("/home/setupishe/datasets/VOC/images", folder))
]

In [8]:
test_list = [
    f"./images/test2007/{x}"
    for x in os.listdir("/home/setupishe/datasets/VOC/images/test2007")
]

In [10]:
with open("/home/setupishe/datasets/VOC/VOC_train.txt", "w") as f:
    f.writelines([x + "\n" for x in train_list])

In [11]:
with open("/home/setupishe/datasets/VOC/VOC_val.txt", "w") as f:
    f.writelines([x + "\n" for x in test_list])

In [2]:
import os, glob, shutil

In [3]:
from PIL import Image

In [4]:
import cv2

In [5]:
def get_shape(img_path):
    with Image.open(img_path) as img:
        width, height = img.size
    return height, width

In [6]:
from tqdm import tqdm

In [7]:
def force_mkdir(directory):
    if os.path.isdir(directory):
        shutil.rmtree(directory)
    os.mkdir(directory)

In [11]:
import os, shutil, glob

In [13]:
def jpg2txt(inp):
    return inp.replace("/images", "/labels").replace(".jpg", ".txt")


def txt2jpg(inp):
    return inp.replace("/labels", "/images").replace(".txt", ".jpg")

In [15]:
from tqdm import tqdm

In [ ]:
for file in tqdm(
    glob.glob("/home/setupishe/datasets/VOC/images/**/*.jpg", recursive=True)
):
    to_folder = "val" if "test2007" in file else "train"
    from_folder_name = file.split("/")[-2]

    for func in [lambda x: x, jpg2txt]:
        to_path = func(file).replace(from_folder_name, to_folder)
        shutil.copy(func(file), to_path)

100%|██████████| 21503/21503 [00:01<00:00, 13256.40it/s]


In [ ]:
for file in glob.glob("/home/setupishe/datasets/VOC/*txt"):
    with open(file, "r") as f:
        lines = f.readlines()
    new_lines = []
    for line in lines:
        from_folder_name = line.split("/")[-2]
        to_folder = "val" if "test2007" in line else "train"
        new_lines.append(line.replace(from_folder_name, to_folder))
    with open(file, "w") as f:
        f.writelines(new_lines)

In [ ]:
with open("/home/setupishe/datasets/VOC/val.txt", "r") as f:
    files = [x.rstrip("\n") for x in f.readlines()]
    for file in files:
        full_path = os.path.join("/home/setupishe/datasets/VOC", file)
        if not os.path.getsize(full_path):
            print(file)

In [5]:
with open("/home/setupishe/datasets/coco/train2017_0.2.txt", "r") as f:
    lines = f.readlines()

In [ ]:
weights = "/home/setupishe/ultralytics/runs/detect/confidences_0.6/weights/best.pt"

conf_path = "/".join(weights.split("/")[:-2])
conf_path = "/home/setupishe/ultralytics/runs/detect/temp6" + "/best_conf.txt"

In [ ]:
with open(conf_path, "r") as f:
    conf = float(f.readline())

In [7]:
conf

0.46446446446446443

In [6]:
names = [os.path.basename(x)[:-1] for x in lines]

In [9]:
folder_name = "imgs_0.2"

In [10]:
force_mkdir(folder_name)

In [8]:
def get_shape(img_path):
    with Image.open(img_path) as img:
        width, height = img.size
    return height, width

In [9]:
def txt2jpg(inp):
    return inp.replace("/labels", "/images").replace(".txt", ".jpg")

In [10]:
def seg2bbox(filepath, to_path):
    with open(filepath, "r") as f:
        lines = [x.rstrip("\n") for x in f.readlines()]
    new_lines = []
    for line in lines:
        lst = line.split()
        new_line = [lst[0]]

        coords = [float(x) for x in lst[1:]]
        h, w = get_shape(txt2jpg(filepath))
        first = [x for i, x in enumerate(coords) if i % 2 == 0]
        second = [x for i, x in enumerate(coords) if i % 2 == 1]
        for i in range(len(first)):
            first[i] = int(first[i] * w)
            second[i] = int(second[i] * h)

        xmin = min(first)
        xmax = max(first)
        ymin = min(second)
        ymax = max(second)

        width = (xmax - xmin) / w
        height = (ymax - ymin) / h
        xc = (xmax + xmin) / w / 2
        yc = (ymax + ymin) / h / 2

        new_lines.append(
            " ".join([str(item) for item in [lst[0], xc, yc, width, height]]) + "\n"
        )
    with open(to_path, "w") as f:
        f.writelines(new_lines)

In [12]:
filelist = glob.glob("/home/setupishe/datasets/coco/images/train2017/*jpg")
for file in tqdm(filelist):
    label_file = file.replace("images", "labels").replace("jpg", "txt")
    if os.path.exists(label_file):
        seg2bbox(label_file, file.replace("jpg", "txt"))
    else:
        os.mknod(file.replace("txt"))

  0%|          | 0/118287 [00:00<?, ?it/s]

100%|██████████| 118287/118287 [01:10<00:00, 1677.78it/s]


### remainder embeds

In [33]:
folder_name = "remainder_imgs_0.2"

In [34]:
force_mkdir(folder_name)

In [35]:
filelist = glob.glob("/home/setupishe/datasets/coco/images/train2017/*jpg")
for file in tqdm(filelist):
    if os.path.basename(file) not in names:
        shutil.copy(file, os.path.join(folder_name, os.path.basename(file)))
        label_file = file.replace("images", "labels").replace("jpg", "txt")
        if os.path.exists(label_file):
            seg2bbox(
                label_file, os.path.join(folder_name, os.path.basename(label_file))
            )
        else:
            os.mknod(label_file)

100%|██████████| 118287/118287 [01:48<00:00, 1090.82it/s]


запускаем `produce_detection_embeddings.py`

запускаем `preprocess_embedding_pool.py`

In [2]:
for file in glob.glob("remainder_embeds_reduced_0.2/**/*npy", recursive=True):
    if len(os.path.basename(file).split("_")) == 2:
        os.remove(file)

In [ ]:
for file in glob.glob("rembeds_reduced_0.2/**/*npy", recursive=True):
    if len(os.path.basename(file).split("_")) == 2:
        os.remove(file)

запускаем `testing_embeds.ipynb`

In [26]:
uniq = set()
for file in os.listdir("test_folder"):
    uniq.add(file.split("_")[0])

In [27]:
len(uniq)

13332

In [36]:
target_num = 118287 * 0.1

In [37]:
not_bgs = [x + ".jpg" for x in random.sample(list(uniq), int(target_num * (1 - 0.008)))]

ищем свободные не использованные фоны

In [38]:
from tqdm import tqdm

In [39]:
free_bgs = []

In [40]:
for file in tqdm(
    glob.glob("/home/setupishe/datasets/coco/labels/train*/**/*txt", recursive=True)
):
    name = os.path.basename(file)
    if not os.path.getsize(file) and name not in names:
        free_bgs.append(file)

100%|██████████| 118287/118287 [00:00<00:00, 399844.65it/s]


In [41]:
bgs = [
    os.path.basename(x).replace("txt", "jpg")
    for x in random.sample(free_bgs, int(target_num * 0.008))
]

In [42]:
with open("/home/setupishe/datasets/coco/train2017_0.3_active.txt", "w") as f:
    f.writelines([f"./images/train2017/{x}\n" for x in names + not_bgs + bgs])

In [1]:
int_frac = 0.4

In [2]:
with open(
    "/home/setupishe/ultralytics/ultralytics/cfg/datasets/coco.yaml", "r"
) as from_file:
    lines = from_file.readlines()

for i, line in enumerate(lines):
    if "train: train2017.txt" in line:
        lines[i] = lines[i].replace("train2017.txt", f"train2017_{int_frac}_active.txt")
with open(
    f"/home/setupishe/ultralytics/ultralytics/cfg/datasets/coco_{int_frac}_active.yaml",
    "w",
) as to_file:
    to_file.writelines(lines)

In [ ]:
import numpy as np
from scipy.ndimage import binary_dilation
import time


# Define the function for dilation
def dilate_3d_spherical(input_array, radius):
    """
    Dilate a 3D binary array using a spherical kernel of a given radius.

    Parameters:
        input_array (numpy.ndarray): 3D binary array to be dilated.
        radius (int): Radius of the spherical kernel.

    Returns:
        numpy.ndarray: Dilated 3D array.
    """
    # Generate the spherical kernel using efficient broadcasting
    size = 2 * radius + 1
    x, y, z = np.ogrid[:size, :size, :size]
    center = radius
    distance = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)
    structuring_element = distance <= radius

    # Perform the dilation
    dilated_array = binary_dilation(input_array, structure=structuring_element)

    return dilated_array


# Create a large 3D binary array for testing
array_size = (512, 512, 512)  # Size of the array
large_array = np.zeros(array_size, dtype=bool)

# Add some random points to the array to act as seeds for dilation
np.random.seed(42)
num_points = 1000  # Number of random points
random_indices = np.random.randint(0, array_size[0], size=(num_points, 3))
for idx in random_indices:
    large_array[tuple(idx)] = True

# Test the dilation function with a spherical radius
radius = 5

start_time = time.time()
dilated_array = dilate_3d_spherical(large_array, radius)
end_time = time.time()

time_taken = end_time - start_time
time_taken

29.452413320541382